In [ ]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import skimage as sk
import skimage.io as skio
import scipy
import cv2
import lib
from align_image_code import align_images

In [ ]:
# print(np.__version__)
# print(matplotlib.__version__)
# print(sk.__version__)
# print(scipy.__version__)
# print(cv2.__version__)

In [ ]:
# part 1.1
imname = "../images/cameraman.png"
im = skio.imread(imname, as_gray=True)

# lib.display_image(im)

In [ ]:
# finite difference operators
d_x = [[1, -1]]
d_y = [[1], [-1]]

pd_x = scipy.signal.convolve2d(im, d_x, mode='same', boundary='symm')
pd_y = scipy.signal.convolve2d(im, d_y, mode='same', boundary='symm')

# lib.save_file("cameraman_pdx.jpg", sk.img_as_ubyte(pd_x))
# lib.save_file("cameraman_pdy.jpg", sk.img_as_ubyte(pd_y))

# skio.imshow(pd_x, cmap="gray")
# skio.show()
# skio.imshow(pd_y, cmap="gray")
# skio.show()

In [ ]:
# compute gradient magnitude image then binarize to acquire edge image
grad_mag = np.sqrt(np.add(np.square(pd_x), np.square(pd_y)))
grad_mag = np.where(grad_mag > 1.0, 1.0, grad_mag)

threshold = 0.275
binarized_grad_mag = np.where(grad_mag > threshold, 1.0, 0.0)

# lib.save_file("cameraman_grad_mag.jpg", sk.img_as_ubyte(grad_mag))
# lib.save_file("cameraman_binarized_grad_mag.jpg", sk.img_as_ubyte(binarized_grad_mag))

# skio.imshow(grad_mag, cmap="gray")
# skio.show()
# skio.imshow(binarized_grad_mag, cmap="gray")
# skio.show()

In [ ]:
# part 1.2
# create and plot Gaussian filter
def make_gaussian(sigma):
    # "rule of thumb half-width of kernel should be sigma * 3" - prof
    ksize = int(np.ceil(sigma * 3) // 2 * 2 + 1)
    g = cv2.getGaussianKernel(ksize, sigma)
    g_T = np.transpose(g)
    g_filter = np.ma.outer(g, g_T)

    # plot the gaussian
    # x, y = np.meshgrid(range(ksize), range(ksize))
    # fig, ax = plt.subplots(subplot_kw={"projection": "3d"})
    # ax.set_title(f"Gaussian Filter with Sigma = {sigma}, Size = {ksize}")
    # ax.plot_surface(x, y, g_filter, cmap=matplotlib.cm.Reds)
    # plt.show()
    
    return g_filter

In [ ]:
sigma = 2
g_filter = make_gaussian(sigma)

In [ ]:
# blur with gaussian then convolve with finite difference operators
im_blurred = scipy.signal.convolve2d(im, g_filter, mode='full', boundary='wrap')

d_x = [[1, -1]]
d_y = [[1], [-1]]

pd_x = scipy.signal.convolve2d(im_blurred, d_x, mode='same', boundary='symm')
pd_y = scipy.signal.convolve2d(im_blurred, d_y, mode='same', boundary='symm')

# lib.save_file("cameraman_blurred.jpg", sk.img_as_ubyte(im_blurred))

# skio.imshow(im_blurred, cmap="gray")
# skio.show()
# skio.imshow(pd_x, cmap="gray")
# skio.show()
# skio.imshow(pd_y, cmap="gray")
# skio.show()

In [ ]:
grad_mag = np.sqrt(np.add(np.square(pd_x), np.square(pd_y)))
# grad_mag = np.where(grad_mag > 1.0, 1.0, grad_mag)

threshold = 0.06
binarized_grad_mag = np.where(grad_mag > threshold, 1.0, 0.0)

# lib.save_file("cameraman_blurred_grad_mag.jpg", sk.img_as_ubyte(grad_mag))
# lib.save_file("cameraman_blurred_binarized_grad_mag.jpg", sk.img_as_ubyte(binarized_grad_mag))

# skio.imshow(grad_mag, cmap="gray")
# skio.show()
# skio.imshow(binarized_grad_mag, cmap="gray")
# skio.show()

In [ ]:
# convolve the gaussian with finite difference operators then apply derivative to image
d_x = [[1, -1]]
d_y = [[1], [-1]]

gpd_x = scipy.signal.convolve2d(g_filter, d_x, mode='full')
gpd_y = scipy.signal.convolve2d(g_filter, d_y, mode='full')

# skio.imshow(gpd_x, cmap="gray")
# skio.show()
# skio.imshow(gpd_y, cmap="gray")
# skio.show()

In [ ]:
pd_x = scipy.signal.convolve2d(im, gpd_x, mode='same', boundary='symm')
pd_y = scipy.signal.convolve2d(im, gpd_y, mode='same', boundary='symm')

grad_mag = np.sqrt(np.add(np.square(pd_x), np.square(pd_y)))
# grad_mag = np.where(grad_mag > 1.0, 1.0, grad_mag)

threshold = 0.06
binarized_grad_mag = np.where(grad_mag > threshold, 1.0, 0.0)

# lib.save_file("cameraman_blurred_grad_mag_1.jpg", sk.img_as_ubyte(grad_mag))
# lib.save_file("cameraman_blurred_binarized_grad_mag_1.jpg", sk.img_as_ubyte(binarized_grad_mag))

# skio.imshow(grad_mag, cmap="gray")
# skio.show()
# skio.imshow(binarized_grad_mag, cmap="gray")
# skio.show()

In [ ]:
# part 2.1
# taj
_, im = lib.get_file("taj.jpg")
b, g, r = cv2.split(im)
# lib.display_image(im)

In [ ]:
# create gaussian filter
sigma = 2
g_filter = make_gaussian(sigma)

In [ ]:
# showing intermediate steps
alpha = 2

b_blurred = scipy.signal.convolve2d(b, g_filter, mode='same', boundary='wrap')
g_blurred = scipy.signal.convolve2d(g, g_filter, mode='same', boundary='wrap')
r_blurred = scipy.signal.convolve2d(r, g_filter, mode='same', boundary='wrap')

im_blurred = cv2.merge([b_blurred, g_blurred, r_blurred])
im_blurred = sk.img_as_ubyte(im_blurred)
# lib.save_file("taj_blurred.jpg", im_blurred)

b_high = b - b_blurred
g_high = g - g_blurred
r_high = r - r_blurred

im_high = cv2.merge([b_high, g_high, r_high])
im_high = sk.img_as_ubyte(im_high)
im_high = (im_high - np.min(im_high))/np.ptp(im_high)
im_high = (im_high * 255.).astype(np.uint8)
# lib.save_file("taj_high.jpg", im_high)

b_sharp = b + b_high * alpha
g_sharp = g + g_high * alpha
r_sharp = r + r_high * alpha

im_out = cv2.merge([b_sharp, g_sharp, r_sharp])

# lib.display_image(im)
# lib.display_image(im_blurred)
# lib.display_image(im_high)
# lib.display_image(im_out)

In [ ]:
# creating the unsharp mask filter
# convolve im with the unsharp mask filter with parameters g as the Gaussian to sharpen it
def sharpen(im, alpha, g):
    # unit impulse, same size as the Gaussian
    e = np.zeros(np.shape(g))
    e[len(e) // 2, len(e[0]) // 2] = 1
    
    filter = (1 + alpha) * e - alpha * g
    im_out = scipy.signal.convolve2d(im, filter, mode='same', boundary='wrap')
    
    return im_out

In [ ]:
# same process but on my own image
_, im = lib.get_file("forbidden_city.jpg")
b, g, r = cv2.split(im)
# lib.display_image(im)

# create gaussian filter
sigma = 2
g_filter = make_gaussian(sigma)

In [ ]:
# showing intermediate steps
alpha = 2

b_blurred = scipy.signal.convolve2d(b, g_filter, mode='same', boundary='wrap')
g_blurred = scipy.signal.convolve2d(g, g_filter, mode='same', boundary='wrap')
r_blurred = scipy.signal.convolve2d(r, g_filter, mode='same', boundary='wrap')

im_blurred = cv2.merge([b_blurred, g_blurred, r_blurred])
im_blurred = sk.img_as_ubyte(im_blurred)
# lib.save_file("fcity_blurred.jpg", im_blurred)

b_high = b - b_blurred
g_high = g - g_blurred
r_high = r - r_blurred

im_high = cv2.merge([b_high, g_high, r_high])
im_high = sk.img_as_ubyte(im_high)
im_high = (im_high - np.min(im_high))/np.ptp(im_high)
im_high = (im_high * 255.).astype(np.uint8)
# lib.save_file("fcity_high.jpg", im_high)

b_sharp = b + b_high * alpha
g_sharp = g + g_high * alpha
r_sharp = r + r_high * alpha

im_out = cv2.merge([b_sharp, g_sharp, r_sharp])

# lib.display_image(im)
# lib.display_image(im_blurred)
# lib.display_image(im_high)
# lib.display_image(im_out)

In [ ]:
# all in one step
alpha = 2

b_sharp = sharpen(b, alpha, g_filter)
g_sharp = sharpen(g, alpha, g_filter)
r_sharp = sharpen(r, alpha, g_filter)

im_out = cv2.merge([b_sharp, g_sharp, r_sharp])

# lib.display_image(im_out)

In [ ]:
# blur a sharp image, then sharpen again
# original
_, im = lib.get_file("flowers.jpeg")
b, g, r = cv2.split(im)
# lib.display_image(im)

# create gaussian filter
sigma = 5
g_filter = make_gaussian(sigma)

# sharpen
alpha = 5
b_sharp = sharpen(b, alpha, g_filter)
g_sharp = sharpen(g, alpha, g_filter)
r_sharp = sharpen(r, alpha, g_filter)

im_sharp = cv2.merge([b_sharp, g_sharp, r_sharp])
# lib.display_image(im_sharp)

# blur
b_blurred = scipy.signal.convolve2d(b_sharp, g_filter, mode='same', boundary='wrap')
g_blurred = scipy.signal.convolve2d(g_sharp, g_filter, mode='same', boundary='wrap')
r_blurred = scipy.signal.convolve2d(r_sharp, g_filter, mode='same', boundary='wrap')
im_blurred = cv2.merge([b_blurred, g_blurred, r_blurred])

# lib.display_image(im_blurred)

# extract high frequencies with blurred image
b_high = b_sharp - b_blurred
g_high = g_sharp - g_blurred
r_high = r_sharp - r_blurred

# add by higher frequencies
alpha = 2
b_blurred += b_high * alpha
g_blurred += b_high * alpha
r_blurred += b_high * alpha

im_sharpened = cv2.merge([b_blurred, g_blurred, r_blurred])
# lib.display_image(im_sharpened)

In [ ]:
# part 2.2
# lowpass im1, highpass im2
def make_hybrid(im1, im2, lowpass_sigma, highpass_sigma, alpha=1):
    lowpass_g_filter = make_gaussian(lowpass_sigma)
    im1_lowpass = scipy.signal.convolve2d(im1, lowpass_g_filter, mode='same', boundary='wrap')
    
    highpass_g_filter = make_gaussian(highpass_sigma)
    im2_highpass = im2 - scipy.signal.convolve2d(im2, highpass_g_filter, mode='same', boundary='wrap')
    
    im_hybrid = im1_lowpass + im2_highpass * alpha

    # clipping
    im_hybrid = np.where(im_hybrid > 1.0, 1.0, im_hybrid)
    im_hybrid = sk.img_as_ubyte(im_hybrid)
    
    # skio.imshow(im1_lowpass, cmap="gray")
    # skio.show()
    # skio.imshow(im2_highpass, cmap="gray")
    # skio.show()
    # skio.imshow(im_hybrid, cmap="gray")
    # skio.show()

    return im_hybrid

In [ ]:
# get images and align
_, im1 = lib.get_file("derek.jpg")
_, im2 = lib.get_file("nutmeg.jpg")

%matplotlib tk
im1_aligned, im2_aligned = align_images(im1, im2)
%matplotlib inline

im1_aligned = sk.color.rgb2gray(im1_aligned)
im2_aligned = sk.color.rgb2gray(im2_aligned)

lowpass_sigma = 9.5
highpass_sigma = 8.5
im_hybrid = make_hybrid(im1_aligned, im2_aligned, lowpass_sigma, highpass_sigma)
# lib.save_file("derek_nutmeg.jpg", im_hybrid)

In [ ]:
lowpass_g_filter = make_gaussian(lowpass_sigma)
im1_lowpass = scipy.signal.convolve2d(im1_aligned, lowpass_g_filter, mode='same', boundary='wrap')

highpass_g_filter = make_gaussian(highpass_sigma)
im2_highpass = im2_aligned - scipy.signal.convolve2d(im2_aligned, highpass_g_filter, mode='same', boundary='wrap')

im_hybrid = im1_lowpass + im2_highpass * alpha

# clipping
im_hybrid = np.where(im_hybrid > 1.0, 1.0, im_hybrid)
im_hybrid = sk.img_as_ubyte(im_hybrid)

im1_aligned_freq = np.log(np.abs(np.fft.fftshift(np.fft.fft2(im1_aligned))))
im1_aligned_freq = (im1_aligned_freq - np.min(im1_aligned_freq))/np.ptp(im1_aligned_freq)
im1_aligned_freq = (im1_aligned_freq * 255.).astype(np.uint8)
# lib.save_file("derek_freq.jpg", im1_aligned_freq)
# skio.imshow(im1_aligned_freq)
# skio.show()

im1_filtered_freq = np.log(np.abs(np.fft.fftshift(np.fft.fft2(im1_lowpass))))
im1_filtered_freq = (im1_filtered_freq - np.min(im1_filtered_freq))/np.ptp(im1_filtered_freq)
im1_filtered_freq = (im1_filtered_freq * 255.).astype(np.uint8)
# lib.save_file("derek_filtered_freq.jpg", im1_filtered_freq)
# skio.imshow(im1_filtered_freq)
# skio.show()

im2_aligned_freq = np.log(np.abs(np.fft.fftshift(np.fft.fft2(im2_aligned))))
im2_aligned_freq = (im2_aligned_freq - np.min(im2_aligned_freq))/np.ptp(im2_aligned_freq)
im2_aligned_freq = (im2_aligned_freq * 255.).astype(np.uint8)
# lib.save_file("nutmeg_freq.jpg", im2_aligned_freq)
# skio.imshow(im2_aligned_freq)
# skio.show()

im2_filtered_freq = np.log(np.abs(np.fft.fftshift(np.fft.fft2(im2_highpass))))
im2_filtered_freq = (im2_filtered_freq - np.min(im2_filtered_freq))/np.ptp(im2_filtered_freq)
im2_filtered_freq = (im2_filtered_freq * 255.).astype(np.uint8)
# lib.save_file("nutmeg_filtered_freq.jpg", im2_filtered_freq)
# skio.imshow(im2_filtered_freq)
# skio.show()

im_hybrid_freq = np.log(np.abs(np.fft.fftshift(np.fft.fft2(im_hybrid))))
im_hybrid_freq = (im_hybrid_freq - np.min(im_hybrid_freq))/np.ptp(im_hybrid_freq)
im_hybrid_freq = (im_hybrid_freq * 255.).astype(np.uint8)
# lib.save_file("im_hybrid_freq.jpg", im_hybrid_freq)
# skio.imshow(im_hybrid_freq)
# skio.show()

In [ ]:
_, im1 = lib.get_file("coco2.jpeg")
_, im2 = lib.get_file("coco.jpeg")

%matplotlib tk
im1_aligned, im2_aligned = align_images(im1, im2)
%matplotlib inline

im1_aligned = sk.color.rgb2gray(im1_aligned)
im2_aligned = sk.color.rgb2gray(im2_aligned)

lowpass_sigma = 4
highpass_sigma = 5
im_hybrid = make_hybrid(im1_aligned, im2_aligned, lowpass_sigma, highpass_sigma)
# skio.imshow(im_hybrid, cmap="gray")
# skio.show()
# lib.save_file("coco_sqrt.jpg", im_hybrid)

In [ ]:
_, im1 = lib.get_file("shelf_empty.jpg")
_, im2 = lib.get_file("shelf_full.jpg")

%matplotlib tk
im1_aligned, im2_aligned = align_images(im1, im2)
%matplotlib inline

im1_aligned = sk.color.rgb2gray(im1_aligned)
im2_aligned = sk.color.rgb2gray(im2_aligned)

lowpass_sigma = 7
highpass_sigma = 9
im_hybrid = make_hybrid(im1_aligned, im2_aligned, lowpass_sigma, highpass_sigma)
# lib.save_file("shelf_hybrid.jpg", im_hybrid)

In [ ]:
_, im1 = lib.get_file("sink_empty.jpg")
_, im2 = lib.get_file("sink_full.jpg")

%matplotlib tk
im1_aligned, im2_aligned = align_images(im1, im2)
%matplotlib inline

im1_aligned = sk.color.rgb2gray(im1_aligned)
im2_aligned = sk.color.rgb2gray(im2_aligned)

lowpass_sigma = 7
highpass_sigma = 2.5
im_hybrid = make_hybrid(im1_aligned, im2_aligned, lowpass_sigma, highpass_sigma)
# lib.save_file("sink_hybrid.jpg", im_hybrid)

In [ ]:
_, im1 = lib.get_file("pooh_normal.jpeg")
_, im2 = lib.get_file("pooh_tux.jpeg")

%matplotlib tk
im1_aligned, im2_aligned = align_images(im1, im2)
%matplotlib inline

im1_aligned = sk.color.rgb2gray(im1_aligned)
im2_aligned = sk.color.rgb2gray(im2_aligned)

lowpass_sigma = 6
highpass_sigma = 6
im_hybrid = make_hybrid(im1_aligned, im2_aligned, lowpass_sigma, highpass_sigma)
# lib.save_file("pooh_hybrid.jpg", im_hybrid)

In [ ]:
_, im1 = lib.get_file("anime.jpeg")
_, im2 = lib.get_file("coco.jpeg")

%matplotlib tk
im1_aligned, im2_aligned = align_images(im1, im2)
%matplotlib inline

im1_aligned = sk.color.rgb2gray(im1_aligned)
im2_aligned = sk.color.rgb2gray(im2_aligned)

lowpass_sigma = 9
highpass_sigma = 11
im_hybrid = make_hybrid(im1_aligned, im2_aligned, lowpass_sigma, highpass_sigma, alpha=1.5)
# lib.save_file("anime_coco.jpg", im_hybrid)

In [ ]:
# part 2.3
# gaussian stack
def gaussian_stack(im, sigma, layers=6):
    filter = make_gaussian(sigma)
    gstack = [im]
    
    for i in range(layers):
        im = scipy.signal.convolve2d(im, filter, mode='same', boundary='symm')
        gstack.append(im)

    return gstack

# laplacian_stack
def laplacian_stack(im, sigma, layers=6):
    gstack = gaussian_stack(im, sigma, layers)
    lstack = []

    for i in range(len(gstack)-1):
        l = gstack[i] - gstack[i+1]
        lstack.append(l)

    lstack.append(gstack[-1])

    return lstack

def blend(im1, im2, mask):
    return im1 * mask + im2 * (1 - mask)

def blend_full(im1, im2, mask, sigma, layers=6):
    # laplacian stack for im1
    b, g, r = cv2.split(im1)
    a_bls, a_gls, a_rls = laplacian_stack(b, sigma, layers), laplacian_stack(g, sigma, layers), laplacian_stack(r, sigma, layers)

    # laplacian stack for im1
    b, g, r = cv2.split(im2)
    b_bls, b_gls, b_rls = laplacian_stack(b, sigma, layers), laplacian_stack(g, sigma, layers), laplacian_stack(r, sigma, layers)
    
    # gaussian stack for mask
    mask_gstack = gaussian_stack(mask, sigma, layers)

    layers += 1
    # blend
    blended_b = np.zeros(np.shape(im1)[:-1])
    for i in range(layers):
        blended_b += blend(a_bls[i], b_bls[i], mask_gstack[i])
    
    blended_g = np.zeros(np.shape(im1)[:-1])
    for i in range(layers):
        blended_g += blend(a_gls[i], b_gls[i], mask_gstack[i])
    
    blended_r = np.zeros(np.shape(im1)[:-1])
    for i in range(layers):
        blended_r += blend(a_rls[i], b_rls[i], mask_gstack[i])

    im_out = cv2.merge([blended_b, blended_g, blended_r])
    return im_out

In [ ]:
_, orange = lib.get_file("orange.jpeg")
_, apple = lib.get_file("apple.jpeg")
mask = np.zeros(np.shape(orange)[:-1])
mask[:, :len(mask[0]) // 2] = 1.0

im_out = blend_full(apple, orange, mask, 5, 6)

im_out_normalized = (im_out - np.min(im_out))/np.ptp(im_out)
im_out_normalized = (im_out_normalized * 255.).astype(np.uint8)
# lib.save_file("oraple.jpeg", im_out_normalized);

# skio.imshow(im_out_normalized)
# skio.show()

In [ ]:
# oraple
_, apple = lib.get_file("apple.jpeg")
_, orange = lib.get_file("orange.jpeg")

# laplacian stacks of an apple and orange
b, g, r = cv2.split(apple)
sigma = 3
layers = 6
a_bls, a_gls, a_rls = laplacian_stack(b, sigma, layers), laplacian_stack(g, sigma, layers), laplacian_stack(r, sigma, layers)

# just to save the images
i=0
for im in zip(a_bls, a_gls, a_rls):
    im_out = cv2.merge(im)
    im_out_normalized = (im_out - np.min(im_out))/np.ptp(im_out)
    im_out_normalized = (im_out_normalized * 255.).astype(np.uint8)
    # lib.save_file(f"apple_laplacian_{i}.jpeg", im_out_normalized);
    i += 1
    
    # skio.imshow(im_out_normalized)
    # skio.show()

b, g, r = cv2.split(orange)
o_bls, o_gls, o_rls = laplacian_stack(b, sigma, layers), laplacian_stack(g, sigma, layers), laplacian_stack(r, sigma, layers)

i=0
for im in zip(o_bls, o_gls, o_rls):
    im_out = cv2.merge(im)
    im_out_normalized = (im_out - np.min(im_out))/np.ptp(im_out)
    im_out_normalized = (im_out_normalized * 255.).astype(np.uint8)
    # lib.save_file(f"orange_laplacian_{i}.jpeg", im_out_normalized);
    i += 1
    
    # skio.imshow(im_out_normalized)
    # skio.show()

In [ ]:
_, eye = lib.get_file("eye.jpeg")
_, hand = lib.get_file("hand.jpeg")
mask = skio.imread("../images/hand_eye_mask.jpg", as_gray=True)

im_out = blend_full(eye, hand, mask, 5, 6)

im_out_normalized = (im_out - np.min(im_out))/np.ptp(im_out)
im_out_normalized = (im_out_normalized * 255.).astype(np.uint8)
# lib.save_file("hand_eye.jpeg", im_out_normalized);

# skio.imshow(im_out_normalized)
# skio.show()

In [ ]:
_, coke = lib.get_file("coke.jpeg")
_, heysong = lib.get_file("heysong.jpeg")
mask = np.zeros(np.shape(coke)[:-1])
mask[400:, :] = 1.0

im_out = blend_full(heysong, coke, mask, 5, 6)

im_out_normalized = (im_out - np.min(im_out))/np.ptp(im_out)
im_out_normalized = (im_out_normalized * 255.).astype(np.uint8)
# lib.save_file("heysong_coke.jpeg", im_out_normalized);

# skio.imshow(im_out_normalized)
# skio.show()